In [ ]:
import os
import warnings
warnings.filterwarnings('ignore')
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten

from tensorflow.keras.callbacks import EarlyStopping
n_words = 5000
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.imdb.load_data(num_words=n_words)

es = EarlyStopping(monitor = "val_loss", mode = "min", patience = 5)

EPOCHS = 1000

#https://keras.io/api/datasets/imdb/

In [ ]:
#wyświetl pierwszy element zbioru X_train
X_train[0]

In [ ]:
#wyświetl pierwszy element zbioru y_train
y_train[0]

In [ ]:
#ta komórka słuzy dopełeniu ciągów wyrazów. Sieć ma stałą wielkość pierwszej warstwy. Zadania nie mają stałej 
#długości. Dlatego, każde zdanie, które będzie miało mniej niż 100 wyrazów dopełniamy zerami.

from tensorflow.keras.preprocessing.sequence import pad_sequences

max_words = 100

X_train_pad = pad_sequences(X_train, maxlen=max_words, padding="post")
X_test_pad = pad_sequences(X_test, maxlen=max_words, padding="post")

In [ ]:
#kluczowa jest tu warstwa Embedding, w której ustawiamy wielkość słownika, w ilu wymiarach chcemy umieścić słowa 
#Na koniec ustawiamy jak długie są zdania, my ustaliliśmy, że mają 100 wyrazów.
#"Warstwa Embedding przekształca wejściowe liczby całkowite na wektory o 
#określonym wymiarze. Jest to zasadniczo macierz, gdzie każdy wiersz odpowiada wektorowi 
#osadzenia danego indeksu."


from tensorflow.keras.layers import Embedding, Flatten
model = Sequential()
model.add(Embedding(input_dim = n_words, output_dim = 16, input_length = max_words))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.summary()
model.compile(optimizer='adam',loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(X_train_pad,y_train, epochs = EPOCHS, validation_data = (X_test_pad,y_test), callbacks = [es])

In [ ]:
hist = pd.DataFrame(history.history)
plt.figure(figsize=(10,5))
plt.subplot(1,2,1)
plt.plot(hist.loss, label = "loss")
plt.plot(hist.val_loss, label = "val_loss")
plt.xlabel("epoch")
plt.ylabel("loss")
plt.legend()

plt.subplot(1,2,2)
plt.plot(hist.accuracy, label = "acc")
plt.plot(hist.val_accuracy, label = "val_acc")
plt.xlabel("epoch")
plt.ylabel("accuracy")
plt.legend()